In [1]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from torchinfo import summary
import pandas as pd
import os

c:\Users\s8gre\AppData\Local\Programs\Python\Python39\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [2]:
data_path = './data/spectrograms' #looking in subfolder train

yes_no_dataset = datasets.ImageFolder(
    root=data_path,
    transform=transforms.Compose([transforms.Resize((201,81)),
                                  transforms.ToTensor()
                                  ])
)
print(yes_no_dataset)

Dataset ImageFolder
    Number of datapoints: 7985
    Root location: ./data/spectrograms
    StandardTransform
Transform: Compose(
               Resize(size=(201, 81), interpolation=bilinear)
               ToTensor()
           )


In [3]:
class_map=yes_no_dataset.class_to_idx

print("\nClass category and index of the images: {}\n".format(class_map))


Class category and index of the images: {'no': 0, 'yes': 1}



In [4]:
#split data to test and train
#use 80% to train
train_size = int(0.8 * len(yes_no_dataset))
test_size = len(yes_no_dataset) - train_size
yes_no_train_dataset, yes_no_test_dataset = torch.utils.data.random_split(yes_no_dataset, [train_size, test_size])

print("Training size:", len(yes_no_train_dataset))
print("Testing size:",len(yes_no_test_dataset))

Training size: 6388
Testing size: 1597


In [6]:
from collections import Counter

# labels in training set
train_classes = [label for _, label in yes_no_train_dataset]
Counter(train_classes)

Counter({1: 3241, 0: 3147})

In [7]:
train_dataloader = torch.utils.data.DataLoader(
    yes_no_train_dataset,
    batch_size=15,
    num_workers=2,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    yes_no_test_dataset,
    batch_size=15,
    num_workers=2,
    shuffle=True
)

In [8]:
td = train_dataloader.dataset[0][0][0][0]
print(td)

tensor([0.1176, 0.1333, 0.1333, 0.1373, 0.1412, 0.1843, 0.1569, 0.1529, 0.1686,
        0.1961, 0.1529, 0.1333, 0.1922, 0.1490, 0.1176, 0.1216, 0.1255, 0.1255,
        0.1176, 0.1882, 0.1333, 0.2000, 0.1216, 0.1804, 0.1451, 0.1216, 0.1255,
        0.1176, 0.1216, 0.1373, 0.1608, 0.1451, 0.1216, 0.1333, 0.1176, 0.1373,
        0.1294, 0.1255, 0.1647, 0.1373, 0.1255, 0.1647, 0.1490, 0.1333, 0.2039,
        0.1569, 0.1843, 0.1333, 0.1843, 0.1529, 0.1373, 0.1843, 0.1373, 0.2118,
        0.1490, 0.1725, 0.1765, 0.1373, 0.1490, 0.1490, 0.1451, 0.1529, 0.1451,
        0.1882, 0.1804, 0.1529, 0.1765, 0.1961, 0.1922, 0.1490, 0.1490, 0.1804,
        0.1333, 0.1804, 0.1490, 0.1294, 0.1529, 0.1490, 0.1255, 0.1451, 0.1294])


In [10]:
import NeuronalNetwork as CNNet
model = CNNet().to(CNNet.device)

In [11]:
# cost function used to determine best parameters
cost = torch.nn.CrossEntropyLoss()

# used to create optimal parameters
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Create the training function

def train(dataloader, model, loss, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, Y) in enumerate(dataloader):
        
        X, Y = X.to(CNNet.device), Y.to(CNNet.device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')


# Create the validation/test function

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, Y) in enumerate(dataloader):
            X, Y = X.to(CNNet.device), Y.to(CNNet.device)
            pred = model(X)

            test_loss += cost(pred, Y).item()
            correct += (pred.argmax(1)==Y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size

    print(f'\nTest Error:\nacc: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f}\n')

In [17]:
epochs = 1

for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train(train_dataloader, model, cost, optimizer)
    test(test_dataloader, model)
print('Done!')

Epoch 1
-------------------------------
loss: 0.076016  [    0/ 6388]
loss: 0.047809  [ 1500/ 6388]
loss: 0.098754  [ 3000/ 6388]
loss: 0.069761  [ 4500/ 6388]
loss: 0.228786  [ 6000/ 6388]

Test Error:
acc: 95.3%, avg loss: 0.006622

Done!


In [ ]:
summary(model, input_size=(15, 3, 201, 81))

In [ ]:
model.eval()
test_loss, correct = 0, 0
class_map = ['no', 'yes']

with torch.no_grad():
    for batch, (X, Y) in enumerate(test_dataloader):
        print(X)
        X, Y = X.to(CNNet.device), Y.to(CNNet.device)
        pred = model(X)
        print("Predicted:\nvalue={}, class_name= {}\n".format(pred[0].argmax(0),class_map[pred[0].argmax(0)]))
        print("Actual:\nvalue={}, class_name= {}\n".format(Y[0],class_map[Y[0]]))
        break

In [19]:
torch.save(model.state_dict(), "data/model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
